<a href="https://colab.research.google.com/github/yangli2/rl_to_learn/blob/master/mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
from google.colab import drive
import tensorflow_datasets as tfds

GOOGLE_DRIVE = '/content/gdrive'
drive.mount(GOOGLE_DRIVE)
DRIVE_DIR = '{}/My Drive'.format(GOOGLE_DRIVE)
MNIST_DIR = '{}/mnist_dataset'.format(DRIVE_DIR)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
mnist = tfds.image.mnist.MNIST()
mnist.download_and_prepare(download_dir=MNIST_DIR)
mnist_ds = mnist.as_dataset()

In [0]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

class MnistModel(Model):
  def __init__(self):
    super(MnistModel, self).__init__()
    self.conv1 = Conv2D(32, 3, activation='relu')
    self.flatten = Flatten()
    self.d1 = Dense(128, activation='relu')
    self.d2 = Dense(10, activation='softmax')

  def call(self, x):
    x = self.conv1(x)
    x = self.flatten(x)
    x = self.d1(x)
    return self.d2(x)

# Create an instance of the model
model = MnistModel()
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()
optimizer = tf.keras.optimizers.Adam()

train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

@tf.function
def train_step(images, labels):
  with tf.GradientTape() as tape:
    predictions = model(images)
    loss = loss_object(labels, predictions)
  gradients = tape.gradient(loss, model.trainable_variables)
  optimizer.apply_gradients(zip(gradients, model.trainable_variables))

  train_loss(loss)
  train_accuracy(labels, predictions)

@tf.function
def test_step(images, labels):
  predictions = model(images)
  t_loss = loss_object(labels, predictions)

  test_loss(t_loss)
  test_accuracy(labels, predictions)

In [7]:
EPOCHS = 5
for epoch in range(EPOCHS):
  for example in mnist_ds['train'].batch(32):
    train_step(tf.cast(example['image'], tf.float32) / 255.,
                tf.cast(example['label'], tf.float32))

  for test_example in mnist_ds['test'].batch(32):
    test_step(tf.cast(test_example['image'], tf.float32) / 255.,
              tf.cast(test_example['label'], tf.float32))

  template = 'Epoch {}, Loss: {}, Accuracy: {}, Test Loss: {}, Test Accuracy: {}'
  print(template.format(epoch+1,
                        train_loss.result(),
                        train_accuracy.result()*100,
                        test_loss.result(),
                        test_accuracy.result()*100))

  # Reset the metrics for the next epoch
  train_loss.reset_states()
  train_accuracy.reset_states()
  test_loss.reset_states()
  test_accuracy.reset_states()

Epoch 1, Loss: 0.14121030271053314, Accuracy: 95.75833129882812, Test Loss: 0.07606296241283417, Test Accuracy: 97.53999328613281
Epoch 2, Loss: 0.04207375645637512, Accuracy: 98.71833038330078, Test Loss: 0.06739327311515808, Test Accuracy: 97.93999481201172
Epoch 3, Loss: 0.019686121493577957, Accuracy: 99.41166687011719, Test Loss: 0.05751356855034828, Test Accuracy: 98.33999633789062
Epoch 4, Loss: 0.01299191266298294, Accuracy: 99.55999755859375, Test Loss: 0.05937952920794487, Test Accuracy: 98.31999969482422
Epoch 5, Loss: 0.009138992056250572, Accuracy: 99.67666625976562, Test Loss: 0.06684920936822891, Test Accuracy: 98.4000015258789
